# 04 — Data Drift Monitoring Analysis

## LinkedIn Lead Scoring Model — MLOps Monitoring

This notebook demonstrates **data drift detection** for the LinkedIn Lead Scoring model.
It is a key deliverable for evaluating the operational monitoring capabilities of the ML pipeline.

**Objectives:**
1. Understand the training distribution (reference data)
2. Detect data drift using Evidently AI on synthetic scenarios
3. Analyse prediction drift when input distributions shift
4. Establish monitoring thresholds and a decision framework for retraining

**Model context:**
- XGBoost classifier trained on 1,910 LemList campaign contacts
- 47 features (19 base numeric + 28 one-hot encoded)
- Best F1 score on validation: **0.556**
- Drift detection powered by [Evidently AI](https://www.evidentlyai.com/)

## 1. Setup & Imports

In [ ]:
import sys
import os
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings

# Add src to path for project imports
PROJECT_ROOT = Path(os.getcwd()).parent
sys.path.insert(0, str(PROJECT_ROOT / "src"))

from linkedin_lead_scoring.monitoring.drift import DriftDetector

# Plotting configuration
plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.dpi"] = 100
plt.rcParams["font.size"] = 11
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)  # suppress scipy/numpy division warnings

# Consistent colour palette
REF_COLOR = "#2196F3"    # Blue for reference
PROD_COLOR = "#FF9800"   # Orange for production/scenario
ALERT_COLOR = "#F44336"  # Red for alerts
OK_COLOR = "#4CAF50"     # Green for no-drift

print(f"Project root: {PROJECT_ROOT}")
print("Setup complete.")

In [ ]:
# Load reference data, model, and configuration
reference_df = pd.read_csv(PROJECT_ROOT / "data" / "reference" / "training_reference.csv")
model = joblib.load(PROJECT_ROOT / "model" / "xgboost_model.joblib")

preprocessor_dict = joblib.load(PROJECT_ROOT / "model" / "preprocessor.joblib")
target_encoder = preprocessor_dict["target_encoder"]
te_cols = preprocessor_dict["te_cols"]

with open(PROJECT_ROOT / "model" / "feature_columns.json") as f:
    feature_columns = json.load(f)

with open(PROJECT_ROOT / "model" / "numeric_medians.json") as f:
    numeric_medians = json.load(f)

# Import feature engineering for processing raw production data
from linkedin_lead_scoring.features import preprocess_for_inference

# Initialize drift detector with reference data
detector = DriftDetector(reference_data=reference_df)

print(f"Reference data shape: {reference_df.shape}")
print(f"Number of features:   {len(feature_columns)}")
print(f"Model type:           {type(model).__name__}")
print(f"Target encoder cols:  {te_cols}")
print(f"Numeric medians:      {numeric_medians}")

---

## 2. Understanding the Training Distribution

Before monitoring for drift, we must understand the **reference distribution** -- the statistical
profile of features the model was trained on. Any future data that deviates significantly from
this baseline may indicate that the model's assumptions no longer hold.

We examine:
- **Numeric features**: LLM quality scores, engagement scores, company founding year
- **One-hot encoded features**: seniority level, geography, business type, company size

In [ ]:
# Summary statistics of the reference data
reference_df.describe().round(3)

In [ ]:
# Distribution of key numeric features
key_numeric = ["llm_quality", "llm_engagement", "llm_decision_maker", "companyfoundedon"]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for ax, col in zip(axes, key_numeric):
    reference_df[col].hist(bins=20, ax=ax, color=REF_COLOR, edgecolor="white", alpha=0.85)
    ax.set_title(col, fontsize=12, fontweight="bold")
    ax.set_xlabel("Value")
    ax.set_ylabel("Count")
    ax.axvline(reference_df[col].median(), color="red", linestyle="--", linewidth=1.2, label="Median")
    ax.legend(fontsize=9)

fig.suptitle("Reference Data — Key Numeric Feature Distributions", fontsize=14, fontweight="bold", y=1.03)
plt.tight_layout()
plt.show()

In [ ]:
# Categorical (OHE) feature distributions — grouped by category prefix
ohe_groups = {
    "Seniority (llm_seniority)": [c for c in feature_columns if c.startswith("llm_seniority_")],
    "Geography (llm_geography)": [c for c in feature_columns if c.startswith("llm_geography_")],
    "Business Type (llm_business_type)": [c for c in feature_columns if c.startswith("llm_business_type_")],
    "Company Size": [c for c in feature_columns if c.startswith("companysize_")],
}

fig, axes = plt.subplots(2, 2, figsize=(14, 8))
axes = axes.flatten()

for ax, (group_name, cols) in zip(axes, ohe_groups.items()):
    # Compute proportion of 1s in each OHE column (= category prevalence)
    proportions = reference_df[cols].mean()
    short_labels = [c.split("_", 2)[-1] if "_" in c else c for c in cols]
    bars = ax.bar(short_labels, proportions, color=REF_COLOR, edgecolor="white", alpha=0.85)
    ax.set_title(group_name, fontsize=12, fontweight="bold")
    ax.set_ylabel("Proportion")
    ax.set_ylim(0, 1.0)
    ax.tick_params(axis="x", rotation=30)
    # Add value labels on bars
    for bar, val in zip(bars, proportions):
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.02,
                f"{val:.0%}", ha="center", fontsize=9)

fig.suptitle("Reference Data — Categorical Feature Distributions (OHE Prevalence)",
             fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

---

## 3. Data Drift Detection — Baseline (No Drift)

We begin with a **no-drift baseline** scenario: synthetic data generated from the same
distribution as the training set. This validates that the drift detector does **not produce
false positives** when data is statistically similar to the reference.

In [ ]:
# Load baseline scenario
baseline_df = pd.read_csv(PROJECT_ROOT / "data" / "drift_scenarios" / "no_drift_baseline.csv")
print(f"Baseline data shape: {baseline_df.shape}")

# Run drift detection
baseline_result = detector.detect_data_drift(baseline_df)

print("\n--- Baseline Drift Detection Results ---")
print(f"  Drift detected:    {baseline_result['drift_detected']}")
print(f"  Drifted features:  {baseline_result['drifted_count']} / {baseline_result['total_features']}")
print(f"  Drift share:       {baseline_result['drift_share']:.1%}")
if baseline_result["drifted_features"]:
    print(f"  Drifted columns:   {baseline_result['drifted_features']}")
else:
    print("  Drifted columns:   (none)")

In [ ]:
# Side-by-side distributions: reference vs baseline for key features
compare_features = ["llm_quality", "llm_engagement", "llm_decision_maker"]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, col in zip(axes, compare_features):
    ax.hist(reference_df[col], bins=15, alpha=0.6, color=REF_COLOR, label="Reference", edgecolor="white")
    ax.hist(baseline_df[col], bins=15, alpha=0.6, color=PROD_COLOR, label="Baseline", edgecolor="white")
    ax.set_title(col, fontsize=12, fontweight="bold")
    ax.set_xlabel("Value")
    ax.set_ylabel("Count")
    ax.legend(fontsize=9)

fig.suptitle("Reference vs No-Drift Baseline — Overlapping Distributions",
             fontsize=14, fontweight="bold", y=1.03)
plt.tight_layout()
plt.show()

**Interpretation:** The baseline scenario produces very low drift share and no overall drift alarm.
The distributions overlap closely with the reference, confirming that the detector is well-calibrated
and does not trigger false positives on in-distribution data.

---

## 4. Data Drift Detection — Synthetic Scenarios

We now test four **intentionally drifted** scenarios, each simulating a realistic production shift:

| Scenario | Description |
|----------|-------------|
| **Sector Shift** | New industries (healthcare, government) absent from training data |
| **Seniority Shift** | Mostly junior profiles instead of the mid/senior mix in training |
| **Geography Shift** | Contacts from different geographic regions |
| **Quality Degradation** | Incomplete profiles (missing summaries, low LLM scores) |

For each scenario, we run Evidently's drift detection and examine which features are most affected.

In [ ]:
# Load all drift scenarios
scenario_files = {
    "Sector Shift": "drift_sector_shift.csv",
    "Seniority Shift": "drift_seniority_shift.csv",
    "Geography Shift": "drift_geography_shift.csv",
    "Quality Degradation": "drift_quality_degradation.csv",
}

scenarios = {}
for name, filename in scenario_files.items():
    df = pd.read_csv(PROJECT_ROOT / "data" / "drift_scenarios" / filename)
    scenarios[name] = df
    print(f"Loaded '{name}': {df.shape}")

In [ ]:
# Run drift detection on each scenario and collect results
drift_results = {}
for name, df in scenarios.items():
    result = detector.detect_data_drift(df)
    drift_results[name] = result

# Also include the baseline for comparison
drift_results["No Drift (Baseline)"] = baseline_result

# Summary table
summary_rows = []
for name in ["No Drift (Baseline)"] + list(scenarios.keys()):
    r = drift_results[name]
    summary_rows.append({
        "Scenario": name,
        "Drift Detected": r["drift_detected"],
        "Drifted Features": r["drifted_count"],
        "Total Features": r["total_features"],
        "Drift Share": f"{r['drift_share']:.1%}",
    })

summary_df = pd.DataFrame(summary_rows)
print("=== Data Drift Detection Summary ===\n")
print(summary_df.to_string(index=False))

In [ ]:
# Visualize drift share across scenarios
scenario_names = [r["Scenario"] for r in summary_rows]
drift_shares = [drift_results[name]["drift_share"] for name in
                ["No Drift (Baseline)"] + list(scenarios.keys())]

fig, ax = plt.subplots(figsize=(10, 5))
colors = [OK_COLOR if ds < 0.2 else (PROD_COLOR if ds < 0.5 else ALERT_COLOR)
          for ds in drift_shares]
bars = ax.barh(scenario_names, [ds * 100 for ds in drift_shares], color=colors, edgecolor="white")

# Add threshold lines
ax.axvline(20, color=PROD_COLOR, linestyle="--", linewidth=1.5, label="Investigate (20%)")
ax.axvline(50, color=ALERT_COLOR, linestyle="--", linewidth=1.5, label="Alert / Retrain (50%)")

# Labels on bars
for bar, ds in zip(bars, drift_shares):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height() / 2,
            f"{ds:.1%}", va="center", fontsize=10, fontweight="bold")

ax.set_xlabel("Drift Share (%)", fontsize=12)
ax.set_title("Data Drift Share by Scenario", fontsize=14, fontweight="bold")
ax.legend(loc="lower right", fontsize=10)
ax.set_xlim(0, 100)
plt.tight_layout()
plt.show()

In [ ]:
# Detailed drifted features for each scenario
for name in scenarios.keys():
    result = drift_results[name]
    drifted = result["drifted_features"]
    print(f"\n{'='*60}")
    print(f"  {name}")
    print(f"  Drifted: {result['drifted_count']}/{result['total_features']} ({result['drift_share']:.1%})")
    print(f"{'='*60}")
    if drifted:
        for i, feat in enumerate(drifted[:10], 1):
            print(f"  {i:2d}. {feat}")
        if len(drifted) > 10:
            print(f"  ... and {len(drifted) - 10} more")
    else:
        print("  (no individually drifted features detected)")

In [ ]:
# Visualize one key drifted feature per scenario (overlapping histograms)
# We pick the most interpretable feature for each scenario type
scenario_highlight_features = {
    "Sector Shift": "llm_industry",
    "Seniority Shift": "llm_decision_maker",
    "Geography Shift": "companylocation",
    "Quality Degradation": "llm_quality",
}

fig, axes = plt.subplots(2, 2, figsize=(14, 8))
axes = axes.flatten()

for ax, (scenario_name, feat) in zip(axes, scenario_highlight_features.items()):
    ref_vals = reference_df[feat]
    scen_vals = scenarios[scenario_name][feat]

    ax.hist(ref_vals, bins=15, alpha=0.6, color=REF_COLOR, label="Reference", edgecolor="white")
    ax.hist(scen_vals, bins=15, alpha=0.6, color=PROD_COLOR, label=scenario_name, edgecolor="white")

    ax.set_title(f"{scenario_name}\nFeature: {feat}", fontsize=11, fontweight="bold")
    ax.set_xlabel("Value")
    ax.set_ylabel("Count")
    ax.legend(fontsize=9)

fig.suptitle("Key Drifted Feature per Scenario — Reference vs Scenario",
             fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

### Scenario Interpretations

- **Sector Shift**: The industry-related features (target-encoded `llm_industry`, `industry`, `companyindustry`) drift because the synthetic data introduces sectors (e.g., healthcare, government) that were underrepresented or absent in training. This simulates a business expansion into new verticals.

- **Seniority Shift**: Features like `llm_decision_maker`, `llm_quality`, and the `llm_seniority_*` OHE columns shift drastically. This simulates targeting more junior contacts who have different profile completeness patterns.

- **Geography Shift**: Location-related features (`companylocation`, `location`, `llm_geography_*`) shift as contacts come from different regions. This is common when expanding to new markets.

- **Quality Degradation**: `llm_quality` drops, `has_summary` goes to 0, and `skills_count` decreases. This simulates a scenario where enrichment quality degrades (e.g., LLM API issues, scraping failures).

---

## 5. Prediction Drift Analysis

Data drift does not always translate into prediction drift. A model may be robust to certain
distributional shifts if the affected features are not strongly weighted.

Here we:
1. Generate model predictions (probability scores) for the reference and each scenario
2. Use a **Kolmogorov-Smirnov (KS) test** to compare prediction score distributions
3. Visualize how predicted lead scores shift across scenarios

In [ ]:
# Generate predictions on reference data
# The data is already in the 47-feature format, so we can predict directly
ref_features = reference_df[feature_columns]
ref_scores = model.predict_proba(ref_features)[:, 1]

print(f"Reference prediction scores:")
print(f"  Mean:   {ref_scores.mean():.3f}")
print(f"  Median: {np.median(ref_scores):.3f}")
print(f"  Std:    {ref_scores.std():.3f}")
print(f"  Range:  [{ref_scores.min():.3f}, {ref_scores.max():.3f}]")

In [ ]:
# Prediction drift analysis for each scenario
all_scenario_names = ["No Drift (Baseline)"] + list(scenarios.keys())
all_scenario_data = {"No Drift (Baseline)": baseline_df, **scenarios}

prediction_results = {}
for name in all_scenario_names:
    df = all_scenario_data[name]
    scen_features = df[feature_columns]
    scen_scores = model.predict_proba(scen_features)[:, 1]

    pred_drift = detector.detect_prediction_drift(ref_scores, scen_scores)
    prediction_results[name] = {
        "scores": scen_scores,
        **pred_drift,
    }

# Summary table
pred_rows = []
for name in all_scenario_names:
    r = prediction_results[name]
    pred_rows.append({
        "Scenario": name,
        "KS Statistic": f"{r['statistic']:.4f}",
        "p-value": f"{r['p_value']:.4e}" if r["p_value"] < 0.001 else f"{r['p_value']:.4f}",
        "Drift Detected": r["drift_detected"],
        "Mean Score": f"{r['scores'].mean():.3f}",
    })

pred_summary_df = pd.DataFrame(pred_rows)
print("=== Prediction Drift Summary (KS Test, threshold=0.05) ===\n")
print(pred_summary_df.to_string(index=False))

In [ ]:
# Overlapping histograms of prediction score distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
axes = axes.flatten()

drift_scenario_names = list(scenarios.keys())
for ax, name in zip(axes, drift_scenario_names):
    r = prediction_results[name]
    color = ALERT_COLOR if r["drift_detected"] else PROD_COLOR

    ax.hist(ref_scores, bins=20, alpha=0.5, color=REF_COLOR, label="Reference", edgecolor="white")
    ax.hist(r["scores"], bins=20, alpha=0.5, color=color, label=name, edgecolor="white")

    status = "DRIFT" if r["drift_detected"] else "OK"
    ax.set_title(f"{name}\nKS={r['statistic']:.3f}, p={r['p_value']:.3e} [{status}]",
                 fontsize=10, fontweight="bold")
    ax.set_xlabel("Predicted Probability (engagement)")
    ax.set_ylabel("Count")
    ax.legend(fontsize=9)

fig.suptitle("Prediction Score Distributions — Reference vs Each Scenario",
             fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

### Key Observations

- **Data drift does not always cause prediction drift.** Some feature shifts may affect columns
  the model does not weight heavily.
- **Prediction drift is the stronger signal** for deciding when to retrain, because it directly
  measures whether the model's output behaviour has changed.
- Scenarios with both high data drift *and* prediction drift (p < 0.05) are the most concerning
  and should trigger a retraining investigation.

---

## 6. Real Production Data — Drift Analysis

We now apply drift detection on **real LinkedIn contacts** that were processed through the same
LLM enrichment pipeline (notebook 01) as the training data. These are actual business development
contacts — not synthetic data.

**Data source:** 151 labelled contacts from 3 LemList campaign exports, enriched via gpt-4o-mini,
with known engagement outcomes (47.7% engagement rate).

This is the most important section — it shows whether our model faces **real drift** in production.

In [ ]:
# Load real production data (20-column enriched format, same as training)
production_raw = pd.read_csv(PROJECT_ROOT / "data" / "production" / "new_contacts_enriched.csv")
print(f"Production data: {production_raw.shape}")
print(f"Engagement rate: {production_raw['engaged'].mean():.1%}")
print(f"  Engaged: {production_raw['engaged'].sum()}, Not engaged: {(1 - production_raw['engaged']).sum():.0f}")

# Preprocess to 47-feature format (same as training_reference.csv)
production_features = preprocess_for_inference(
    production_raw.drop(columns=["engaged"]),
    target_encoder=target_encoder,
    te_cols=te_cols,
    feature_columns=feature_columns,
    numeric_medians=numeric_medians,
)
print(f"\nProcessed features: {production_features.shape}")
print(f"Columns match reference: {list(production_features.columns) == feature_columns}")

In [ ]:
# Data drift detection on real production data
prod_drift = detector.detect_data_drift(production_features)

print("=== REAL PRODUCTION DATA — Drift Detection ===\n")
print(f"  Drift detected:    {prod_drift['drift_detected']}")
print(f"  Drift share:       {prod_drift['drift_share']:.1%}")
print(f"  Drifted features:  {prod_drift['drifted_count']} / {prod_drift['total_features']}")
print(f"\n  Drifted feature list:")
for i, feat in enumerate(prod_drift["drifted_features"], 1):
    print(f"    {i:2d}. {feat}")

# Prediction drift on real data
prod_scores = model.predict_proba(production_features[feature_columns])[:, 1]
prod_pred_drift = detector.detect_prediction_drift(ref_scores, prod_scores)

print(f"\n=== Prediction Drift ===")
print(f"  KS Statistic:  {prod_pred_drift['statistic']:.4f}")
print(f"  p-value:       {prod_pred_drift['p_value']:.4e}")
print(f"  Drift:         {prod_pred_drift['drift_detected']}")
print(f"\n  Production scores: mean={prod_scores.mean():.3f}, std={prod_scores.std():.3f}")
print(f"  Reference scores:  mean={ref_scores.mean():.3f}, std={ref_scores.std():.3f}")

In [ ]:
# Visualize real production drift
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Top row: key numeric feature distributions (reference vs production)
top_features = ["llm_quality", "llm_engagement", "llm_decision_maker"]
for ax, col in zip(axes[0], top_features):
    drifted = col in prod_drift["drifted_features"]
    border_color = ALERT_COLOR if drifted else OK_COLOR
    ax.hist(reference_df[col], bins=15, alpha=0.5, color=REF_COLOR, label="Reference (training)", edgecolor="white")
    ax.hist(production_features[col], bins=15, alpha=0.5, color=PROD_COLOR, label="Production (real)", edgecolor="white")
    status = "DRIFTED" if drifted else "OK"
    ax.set_title(f"{col} [{status}]", fontsize=11, fontweight="bold", color=border_color)
    ax.set_xlabel("Value")
    ax.set_ylabel("Count")
    ax.legend(fontsize=9)

# Bottom left: prediction score distributions
ax = axes[1][0]
ax.hist(ref_scores, bins=20, alpha=0.5, color=REF_COLOR, label="Reference", edgecolor="white")
ax.hist(prod_scores, bins=20, alpha=0.5, color=PROD_COLOR, label="Production", edgecolor="white")
drift_label = "DRIFT" if prod_pred_drift["drift_detected"] else "OK"
ax.set_title(f"Prediction Scores [{drift_label}]\nKS={prod_pred_drift['statistic']:.3f}, p={prod_pred_drift['p_value']:.3e}",
             fontsize=11, fontweight="bold")
ax.set_xlabel("Engagement Probability")
ax.set_ylabel("Count")
ax.legend(fontsize=9)

# Bottom middle: drift share comparison (production vs synthetic scenarios)
ax = axes[1][1]
all_names = ["Baseline", "Production\n(REAL)", "Sector", "Seniority", "Geography", "Quality"]
all_shares = [
    baseline_result["drift_share"],
    prod_drift["drift_share"],
    drift_results["Sector Shift"]["drift_share"],
    drift_results["Seniority Shift"]["drift_share"],
    drift_results["Geography Shift"]["drift_share"],
    drift_results["Quality Degradation"]["drift_share"],
]
colors = [OK_COLOR if s < 0.2 else (PROD_COLOR if s < 0.5 else ALERT_COLOR) for s in all_shares]
colors[1] = "#E91E63"  # Highlight production in pink
bars = ax.bar(all_names, [s * 100 for s in all_shares], color=colors, edgecolor="white")
ax.axhline(50, color=ALERT_COLOR, linestyle="--", linewidth=1.5, label="Alert threshold (50%)")
ax.axhline(20, color=PROD_COLOR, linestyle="--", linewidth=1.2, label="Investigate (20%)")
for bar, s in zip(bars, all_shares):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 1, f"{s:.0%}",
            ha="center", fontsize=9, fontweight="bold")
ax.set_ylabel("Drift Share (%)")
ax.set_title("Drift Share Comparison", fontsize=11, fontweight="bold")
ax.legend(fontsize=8)
ax.set_ylim(0, 100)
ax.tick_params(axis="x", rotation=15)

# Bottom right: OHE category comparison
ax = axes[1][2]
ohe_cats = ["companysize_UNKNOWN", "companytype_Privately Held", "companytype_UNKNOWN",
            "llm_seniority_Senior", "llm_business_type_others"]
ref_vals = reference_df[ohe_cats].mean()
prod_vals = production_features[ohe_cats].mean()
x = np.arange(len(ohe_cats))
short_labels = [c.split("_", 1)[-1][:20] for c in ohe_cats]
ax.bar(x - 0.2, ref_vals, 0.35, color=REF_COLOR, label="Reference", edgecolor="white")
ax.bar(x + 0.2, prod_vals, 0.35, color=PROD_COLOR, label="Production", edgecolor="white")
ax.set_xticks(x)
ax.set_xticklabels(short_labels, fontsize=8, rotation=30, ha="right")
ax.set_ylabel("Proportion")
ax.set_title("Key Category Shifts", fontsize=11, fontweight="bold")
ax.legend(fontsize=9)

fig.suptitle("REAL PRODUCTION DATA — Drift Analysis (151 contacts)",
             fontsize=15, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

---

## 7. Concept Drift — Ground Truth Analysis

**Concept drift** differs from data drift: the **relationship between features and the target**
may have changed, even if feature distributions are stable.

We have **151 labelled contacts** with actual engagement outcomes. This allows us to:
1. Compare the model's predictions against ground truth
2. Measure real-world F1, precision, recall on unseen data
3. Determine whether concept drift has occurred (F1 drop from training baseline of 0.556)

This is the strongest evidence for or against retraining.

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report

# Ground truth from the production data
actuals = production_raw["engaged"].values
predictions_proba = prod_scores
predictions_binary = (predictions_proba >= 0.5).astype(int)

# Performance metrics
f1 = f1_score(actuals, predictions_binary)
precision = precision_score(actuals, predictions_binary)
recall = recall_score(actuals, predictions_binary)

print("=== MODEL PERFORMANCE ON REAL PRODUCTION DATA ===\n")
print(f"  Contacts evaluated:  {len(actuals)}")
print(f"  Actual engaged:      {actuals.sum()} ({actuals.mean():.1%})")
print(f"  Predicted engaged:   {predictions_binary.sum()} ({predictions_binary.mean():.1%})")
print()
print(f"  {'Metric':<15} {'Training':>10} {'Production':>12} {'Change':>10}")
print(f"  {'-'*47}")
print(f"  {'F1 Score':<15} {'0.556':>10} {f1:>12.3f} {(f1 - 0.556) / 0.556:>+10.1%}")
print(f"  {'Precision':<15} {'—':>10} {precision:>12.3f}")
print(f"  {'Recall':<15} {'—':>10} {recall:>12.3f}")

# Concept drift assessment
f1_drop = (0.556 - f1) / 0.556
if f1_drop > 0.20:
    print(f"\n  ⚠️  ALERT: F1 dropped {f1_drop:.0%} from training baseline.")
    print(f"     This indicates concept drift. Retraining is recommended.")
elif f1_drop > 0.10:
    print(f"\n  ⚠️  WARNING: F1 dropped {f1_drop:.0%}. Monitor closely.")
else:
    print(f"\n  ✅ F1 is within acceptable range of training performance.")

In [ ]:
# Confusion matrix and detailed classification report
print("=== Confusion Matrix ===\n")
cm = confusion_matrix(actuals, predictions_binary)
print(f"                    Predicted")
print(f"                 Not Eng.  Engaged")
print(f"  Actual Not Eng.  {cm[0][0]:>5}    {cm[0][1]:>5}")
print(f"  Actual Engaged   {cm[1][0]:>5}    {cm[1][1]:>5}")
print()
print(classification_report(actuals, predictions_binary, target_names=["Not Engaged", "Engaged"]))

# Visualization: confusion matrix + score distribution by actual label
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Confusion matrix heatmap
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1,
            xticklabels=["Not Engaged", "Engaged"],
            yticklabels=["Not Engaged", "Engaged"])
ax1.set_xlabel("Predicted", fontsize=12)
ax1.set_ylabel("Actual", fontsize=12)
ax1.set_title("Confusion Matrix — Production Data", fontsize=13, fontweight="bold")

# Score distribution by actual label
engaged_scores = predictions_proba[actuals == 1]
not_engaged_scores = predictions_proba[actuals == 0]
ax2.hist(not_engaged_scores, bins=15, alpha=0.6, color="#F44336", label="Actually Not Engaged", edgecolor="white")
ax2.hist(engaged_scores, bins=15, alpha=0.6, color="#4CAF50", label="Actually Engaged", edgecolor="white")
ax2.axvline(0.5, color="black", linestyle="--", linewidth=2, label="Decision threshold (0.5)")
ax2.set_xlabel("Predicted Engagement Probability", fontsize=12)
ax2.set_ylabel("Count", fontsize=12)
ax2.set_title("Score Distribution by Actual Outcome", fontsize=13, fontweight="bold")
ax2.legend(fontsize=10)

plt.tight_layout()
plt.show()

print(f"\nKey insight: The model scores actually-engaged contacts at {engaged_scores.mean():.3f} on average,")
print(f"vs {not_engaged_scores.mean():.3f} for not-engaged. Separation exists but is weaker than training.")

---

## 8. Monitoring Recommendations & Decision Framework

### Summary of Findings

| Scenario | Data Drift Share | Prediction Drift | Action |
|----------|:----------------:|:-----------------:|--------|
| No-Drift Baseline | ~2% | No | Monitor (all clear) |
| Sector Shift | High | Check results above | Investigate if new verticals are permanent |
| Seniority Shift | High | Check results above | Investigate targeting strategy change |
| Geography Shift | Moderate-High | Check results above | Assess expansion impact |
| Quality Degradation | High | Check results above | Alert -- fix data pipeline first |

### Decision Framework: When to Retrain

| Metric | Threshold | Action |
|--------|-----------|--------|
| **Drift share** < 20% | Green | Continue monitoring. Normal variation. |
| **Drift share** 20-50% | Yellow | Investigate which features drifted. If business-relevant, plan retraining. |
| **Drift share** >= 50% | Red | Alert. Consider immediate retraining or rollback. |
| **Prediction drift** p-value < 0.01 | Red | Model output distribution has shifted significantly. Evaluate real-world impact. |
| **F1 on holdout** drops > 20% from baseline (0.556) | Red | Concept drift confirmed. Retrain with new labelled data. |

### Operational Monitoring Stack

- **Streamlit Dashboard**: Real-time drift metrics and prediction distributions.
  Deployed on Hugging Face Spaces for continuous visibility.
- **Evidently Reports**: Full HTML drift reports generated via `DriftDetector.generate_report()`.
  Stored per batch for audit trail.
- **Retraining Script**: `scripts/export_model.py` re-exports the model with updated data.
  MLflow tracks experiment versions and model registry manages production deployment.

### Next Steps

1. Deploy monitoring dashboard to production
2. Establish ground truth feedback loop (record outreach outcomes)
3. Set up automated drift detection on weekly batches
4. Define SLA: drift report generated within 24h of new batch ingestion

In [ ]:
# Final consolidated summary — combined data drift + prediction drift
print("=" * 80)
print("  CONSOLIDATED DRIFT MONITORING REPORT")
print("=" * 80)
print(f"\n  Reference data:  {reference_df.shape[0]} samples, {len(feature_columns)} features")
print(f"  Model:           XGBoost (F1=0.556 on validation)")
print(f"  Drift detector:  Evidently AI (KS test for numeric, chi-squared for categorical)")
print()

for name in all_scenario_names:
    dr = drift_results[name]
    pr = prediction_results[name]

    # Determine alert level
    if dr["drift_share"] >= 0.5 or pr["drift_detected"]:
        level = "RED"
    elif dr["drift_share"] >= 0.2:
        level = "YELLOW"
    else:
        level = "GREEN"

    print(f"  --- {name} [{level}] ---")
    print(f"    Data drift:       {dr['drifted_count']}/{dr['total_features']} features ({dr['drift_share']:.1%})")
    print(f"    Prediction drift: KS={pr['statistic']:.4f}, p={pr['p_value']:.4e}, detected={pr['drift_detected']}")
    print(f"    Mean score:       {pr['scores'].mean():.3f} (ref: {ref_scores.mean():.3f})")
    print()

print("=" * 80)
print("  End of drift monitoring analysis.")
print("=" * 80)